# Laboratório: Construindo um Agente

Neste notebook, exploraremos como criar um agente inteligente que é capaz de realizar operações matemáticas básicas. O agente utilizará o modelo GPT-4o da OpenAI e será equipado com ferramentas para adição, subtração, multiplicação e divisão.

In [ ]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.openai_functions_agent.base import create_openai_functions_agent
from langchain.agents.agent import AgentExecutor

from langchain.tools import BaseTool
from typing import Type, Any
from pydantic import BaseModel, Field

## Configuração do Ambiente

Carregamos as variáveis de ambiente do arquivo .env, que deve conter a chave da API da OpenAI.

In [2]:
load_dotenv()

True

## Definição das Ferramentas Matemáticas

Aqui, definimos as quatro operações matemáticas básicas como ferramentas que nosso agente poderá utilizar:

1. **AdditionTool**: Para somar dois números
2. **SubtractionTool**: Para subtrair um número de outro
3. **MultiplicationTool**: Para multiplicar dois números
4. **DivisionTool**: Para dividir um número por outro (com verificação de divisão por zero)

Cada ferramenta utiliza o schema `CalculatorInput` para validar seus parâmetros de entrada.

In [3]:
class CalculatorInput(BaseModel):
    a: float = Field(..., description="Primeiro número para a operação")
    b: float = Field(..., description="Segundo número para a operação")

class AdditionTool(BaseTool):
    name: str = "addition"
    description: str = "Útil para somar dois números."
    args_schema: Type[BaseModel] = CalculatorInput
    
    def _run(self, a: float, b: float) -> float:
        """Soma dois números."""
        return a + b

class SubtractionTool(BaseTool):
    name: str = "subtraction"
    description: str = "Útil para subtrair um número de outro."
    args_schema: Type[BaseModel] = CalculatorInput
    
    def _run(self, a: float, b: float) -> float:
        """Subtrai b de a."""
        return a - b

class MultiplicationTool(BaseTool):
    name: str = "multiplication"
    description: str = "Útil para multiplicar dois números."
    args_schema: Type[BaseModel] = CalculatorInput
    
    def _run(self, a: float, b: float) -> float:
        """Multiplica dois números."""
        return a * b

class DivisionTool(BaseTool):
    name: str = "division"
    description: str = "Útil para dividir um número por outro. Retorna erro se o divisor for zero."
    args_schema: Type[BaseModel] = CalculatorInput
    
    def _run(self, a: float, b: float) -> float:
        """Divide a por b."""
        if b == 0:
            raise ZeroDivisionError("Não é possível dividir por zero")
        return a / b

## Criação do Agente Matemático

A função `create_math_agent()` configura um agente LangChain que:

1. Utiliza as ferramentas matemáticas definidas anteriormente
2. Configura o modelo de linguagem GPT-4o como base
3. Define um prompt específico que orienta o modelo a atuar como um assistente matemático
4. Cria um executor de agente com configurações para verbosidade e tratamento de erros

In [ ]:
def create_math_agent():
    tools = [
        AdditionTool(),
        SubtractionTool(),
        MultiplicationTool(),
        DivisionTool()
    ]
    
    llm = ChatOpenAI(
        temperature=0,
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY")
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", """Você é um assistente matemático especializado em resolver problemas passo a passo.
        Analise cuidadosamente cada problema e use a ferramenta mais apropriada para resolvê-lo.
        Explique seu raciocínio de forma clara e detalhada para ajudar o usuário a entender a solução.
        """),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    agent = create_openai_functions_agent(
        llm=llm,
        tools=tools,
        prompt=prompt
    )
    
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        max_iterations=5,
        handle_parsing_errors=True
    )
    
    return agent_executor

## Instanciação do Agente

Vamos criar uma instância do agente matemático.

In [21]:
agent = create_math_agent()

## Exemplos de Uso do Agente

Abaixo, testaremos o agente com diferentes tipos de problemas matemáticos.

### Exemplo 1: Divisão

Vamos testar a capacidade do agente de realizar divisões.

In [22]:
response = agent.invoke({"input": "Se eu dividir 144 por 12, qual é o resultado?"})
print("Response: ", response)



> Entering new AgentExecutor chain...

Invoking: `division` with `{'a': 144, 'b': 12}`


12.0O resultado da divisão de 144 por 12 é 12.

> Finished chain.
Response:  {'input': 'Se eu dividir 144 por 12, qual é o resultado?', 'output': 'O resultado da divisão de 144 por 12 é 12.'}


### Exemplo 2: Adição

Vamos testar a capacidade do agente de realizar somas.

In [23]:
response = agent.invoke({"input": "Qual é o resultado de 15 + 27?"})
print("Response: ", response)



> Entering new AgentExecutor chain...

Invoking: `addition` with `{'a': 15, 'b': 27}`


42.0O resultado de \(15 + 27\) é 42.

> Finished chain.
Response:  {'input': 'Qual é o resultado de 15 + 27?', 'output': 'O resultado de \\(15 + 27\\) é 42.'}


### Exemplo 3: Subtração em Contexto

Vamos testar a capacidade do agente de entender problemas de subtração em contexto.

In [24]:
response = agent.invoke({"input": "Se eu tenho 100 e gasto 35, quanto me resta?"})
print("Response: ", response)



> Entering new AgentExecutor chain...

Invoking: `subtraction` with `{'a': 100, 'b': 35}`


65.0Se você tem 100 e gasta 35, restam 65.

> Finished chain.
Response:  {'input': 'Se eu tenho 100 e gasto 35, quanto me resta?', 'output': 'Se você tem 100 e gasta 35, restam 65.'}


### Exemplo 4: Multiplicação

Vamos testar a capacidade do agente de realizar multiplicações.

In [25]:
response = agent.invoke({"input": "Qual é o produto de 12 e 8?"})
print("Response: ", response)



> Entering new AgentExecutor chain...

Invoking: `multiplication` with `{'a': 12, 'b': 8}`


96.0O produto de 12 e 8 é 96.

> Finished chain.
Response:  {'input': 'Qual é o produto de 12 e 8?', 'output': 'O produto de 12 e 8 é 96.'}


### Exemplo 5: Expressão com Ordem de Operações

Este exemplo testa como o agente lida com expressões que envolvem a ordem das operações matemáticas.

In [26]:
response = agent.invoke({"input": "Resolva 2 + 3 * 4"})
print("Response: ", response)



> Entering new AgentExecutor chain...
Para resolver a expressão \(2 + 3 \times 4\), devemos seguir a ordem das operações matemáticas, que é:

1. Parênteses
2. Expoentes
3. Multiplicação e Divisão (da esquerda para a direita)
4. Adição e Subtração (da esquerda para a direita)

Na expressão dada, temos uma multiplicação e uma adição. De acordo com a ordem das operações, devemos realizar a multiplicação antes da adição.

Primeiro, calculamos \(3 \times 4\):

\[3 \times 4 = 12\]

Agora, substituímos o resultado na expressão original:

\[2 + 12\]

Por fim, realizamos a adição:

\[2 + 12 = 14\]

Portanto, o resultado da expressão \(2 + 3 \times 4\) é 14.

> Finished chain.
Response:  {'input': 'Resolva 2 + 3 * 4', 'output': 'Para resolver a expressão \\(2 + 3 \\times 4\\), devemos seguir a ordem das operações matemáticas, que é:\n\n1. Parênteses\n2. Expoentes\n3. Multiplicação e Divisão (da esquerda para a direita)\n4. Adição e Subtração (da esquerda para a direita)\n\nNa expressão dada, t

In [27]:
response = agent.invoke({"input": "Resolva a operação matemática: Some 32 com 97 e depois multiplique por 12, faça a divisão do resultado por 4 e subtraia 10"})
print("Response: ", response)



> Entering new AgentExecutor chain...

Invoking: `addition` with `{'a': 32, 'b': 97}`


129.0
Invoking: `multiplication` with `{'a': 129, 'b': 12}`


1548.0
Invoking: `division` with `{'a': 1548, 'b': 4}`


387.0
Invoking: `subtraction` with `{'a': 387, 'b': 10}`


377.0Vamos resolver a operação passo a passo:

1. Some 32 com 97:
   \[
   32 + 97 = 129
   \]

2. Multiplique o resultado por 12:
   \[
   129 \times 12 = 1548
   \]

3. Divida o resultado por 4:
   \[
   \frac{1548}{4} = 387
   \]

4. Subtraia 10 do resultado:
   \[
   387 - 10 = 377
   \]

Portanto, o resultado final da operação é 377.

> Finished chain.
Response:  {'input': 'Resolva a operação matemática: Some 32 com 97 e depois multiplique por 12, faça a divisão do resultado por 4 e subtraia 10', 'output': 'Vamos resolver a operação passo a passo:\n\n1. Some 32 com 97:\n   \\[\n   32 + 97 = 129\n   \\]\n\n2. Multiplique o resultado por 12:\n   \\[\n   129 \\times 12 = 1548\n   \\]\n\n3. Divida o resultado por 4:\n   \

## Conclusão

Neste laboratório, construímos um agente matemático capaz de:

1. Entender problemas matemáticos em linguagem natural
2. Selecionar a ferramenta apropriada para cada operação
3. Executar cálculos com precisão
4. Fornecer explicações detalhadas dos passos realizados

O agente demonstra como os modelos de linguagem podem ser aprimorados com ferramentas específicas para realizar tarefas estruturadas e seguir regras matemáticas corretamente.